In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize the Gemini model
# Assuming your Gemini API key is stored in an environment variable named GOOGLE_API_KEY
gemini_api_key = os.getenv("GOOGLE_API_KEY")
if not gemini_api_key:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api_key)

# Create a prompt template for summarizing the reflection
reflection_prompt_template = PromptTemplate(
    input_variables=["reflection"],
    template="""Please provide a one-line summary of the following reflection:

{reflection}""",
)

# Create a Langchain chain for summarizing the reflection
reflection_summary_chain = LLMChain(llm=llm, prompt=reflection_prompt_template, output_key="reflection_summary")

# Create a prompt template for summarizing the reflection summary
summary_summary_prompt_template = PromptTemplate(
    input_variables=["reflection_summary"],
    template="""Please provide a one-line summary of the following summary:

{reflection_summary}""",
)

# Create a Langchain chain for summarizing the reflection summary
summary_summary_chain = LLMChain(llm=llm, prompt=summary_summary_prompt_template, output_key="summary_summary")

def create_reflection_summary(reflection_data: dict) -> tuple[str, str]:
    """
    Takes a dictionary of reflection questions and answers and provides a one-line summary
    of the reflection and a one-line summary of the generated summary.

    Args:
        reflection_data: A dictionary where keys are questions and values are answers.

    Returns:
        A tuple containing the one-line summary of the reflection and the one-line summary
        of the generated summary.
    """
    reflection_text = "\n".join([f"Question: {q}\nAnswer: {a}" for q, a in reflection_data.items()])

    # Generate the one-line summary of the reflection
    reflection_summary_output = reflection_summary_chain.run(reflection=reflection_text)
    reflection_summary = reflection_summary_output.strip()

    # Generate a one-line summary of the reflection summary
    summary_summary_output = summary_summary_chain.run(reflection_summary=reflection_summary)
    summary_summary = summary_summary_output.strip()

    return reflection_summary, summary_summary

if __name__ == "__main__":
    user_reflection = {
        "What did you learn today?": "I learned about using Langchain with the Gemini model and environment variables.",
        "What went well?": "Successfully integrated dotenv for API key management.",
        "What could have gone better?": "Ensuring the .env file is properly configured.",
        "What will you do differently next time?": "Explore more robust environment variable management strategies.",
    }

    reflection_summary, summary_summary = create_reflection_summary(user_reflection)

    print("Original Reflection:")
    for question, answer in user_reflection.items():
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print("-" * 20)

    print("\nOne-Line Summary of Reflection:")
    print(reflection_summary)

    print("\nOne-Line Summary of the Summary:")
    print(summary_summary)

/tmp/ipykernel_1427/425629907.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  reflection_summary_chain = LLMChain(llm=llm, prompt=reflection_prompt_template, output_key="reflection_summary")
/tmp/ipykernel_1427/425629907.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  reflection_summary_output = reflection_summary_chain.run(reflection=reflection_text)


Original Reflection:
Question: What did you learn today?
Answer: I learned about using Langchain with the Gemini model and environment variables.
--------------------
Question: What went well?
Answer: Successfully integrated dotenv for API key management.
--------------------
Question: What could have gone better?
Answer: Ensuring the .env file is properly configured.
--------------------
Question: What will you do differently next time?
Answer: Explore more robust environment variable management strategies.
--------------------

One-Line Summary of Reflection:
Today I learned to use Langchain with Gemini and environment variables, successfully integrating dotenv but needing to improve .env file configuration and explore more robust environment variable management.

One-Line Summary of the Summary:
Successfully integrated Langchain with Gemini and environment variables using dotenv, but needs improvement in .env file configuration and more robust environment variable management.
